In [1]:
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus
import numpy as np
from scipy.linalg import lstsq
import pybullet as p
import pybullet_data
import numpy as np

follower_port = "/dev/ttyUSB1"

follower_arm = DynamixelMotorsBus(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera

robot = ManipulatorRobot(
    robot_type="koch",
    follower_arms={"main": follower_arm},
    calibration_dir=".cache/calibration/koch",
    cameras={
        "view1": OpenCVCamera(12, fps=30, width=640, height=480),
    },
)
robot.connect()
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.ENABLED.value)
import time
from lerobot.scripts.control_robot import busy_wait

record_time_s = 1
fps = 60

states = []
actions = []


base_points_camera = np.array([
    [0.10544403, -0.18065751, 0.55699998],  # 第一个基座点
    [0.09391508, -0.16818333, 0.55000001],  # 第二个基座点
    [0.11092141, -0.16563238, 0.55000001]   # 第三个基座点
])

target_position_camera = np.array([-0.024, 0.0381, 0.425])


def calculate_local_coordinate_system(base_points_camera):
    p1, p2, p3 = base_points_camera

    z_axis = np.cross(p2 - p1, p3 - p1)
    z_axis /= np.linalg.norm(z_axis) 

    midpoint = (p1 + p3) / 2
    y_axis = p2 - midpoint
    y_axis /= np.linalg.norm(y_axis) 

    x_axis = np.cross(y_axis, z_axis)
    x_axis /= np.linalg.norm(x_axis)  

    rotation_matrix = np.vstack([x_axis, y_axis, z_axis]).T

    return rotation_matrix

rotation_matrix = calculate_local_coordinate_system(base_points_camera)

base_point_robot = np.array([0, 0, 0])
translation_vector = base_point_robot - rotation_matrix.dot(base_points_camera[0])

def transform_target_position(target_position_camera, rotation_matrix, translation_vector):
    target_position_robot = rotation_matrix.dot(target_position_camera) + translation_vector
    return target_position_robot

target_position_robot = transform_target_position(target_position_camera, rotation_matrix, translation_vector)
print("Target Position in Robot Coordinate System:\n", target_position_robot)
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
robot_id = p.loadURDF("/home/rhos/Desktop/LeapTele/DexCopilot/assets/low_cost_robot_description/urdf/low_cost_robot.urdf", useFixedBase=True)
visual_shape_id = p.createVisualShape(
    shapeType=p.GEOM_SPHERE,
    radius=0.01,  
    rgbaColor=[1, 0, 0, 1]  
)

p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=visual_shape_id,
    basePosition=target_position_robot
)

for _ in range(10000):
    p.stepSimulation()

pybullet build time: Nov 28 2023 23:45:17


Connecting main follower arm.
Activating torque on main follower arm.


/home/robot/miniconda3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Target Position in Robot Coordinate System:
 [-0.16006234  0.23675912  0.0189676 ]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4070/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 550.54.15
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 550.54.15
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4070/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [2]:
def sim2real(joint_angles):
    joint_limits = [
        (-1.5708, 1.5708),  # joint1
        (-0.5236, 1.5708),  # joint2
        (-1.3963, 1.5708),  # joint3
        (0, 3.1416),        # joint4
        (0, 3.1416),        # joint5
        (-1.5708, 0)        # joint_gripper
    ]
    scaled_joint_angles = []
    for i, angle in enumerate(joint_angles):
        lower_limit, upper_limit = joint_limits[i]
        scaled_angle = max(lower_limit, min(upper_limit, angle))
        scaled_joint_angles.append(scaled_angle)

    joint_angles_degrees = np.degrees(joint_angles)
    joint_angles_real= 90 - joint_angles_degrees
    joint_angles_real[0] -= 90

    return joint_angles_real

In [3]:
# ik calculate joints

joint_angles = p.calculateInverseKinematics(robot_id, 5, target_position_robot)
print("Joint Angles in Robot Coordinate System:\n", joint_angles)
joint_limits = [
    (-1.5708, 1.5708),  # joint1
    (-0.5236, 1.5708),  # joint2
    (-1.3963, 1.5708),  # joint3
    (0, 3.1416),        # joint4
    (0, 3.1416),        # joint5
    (-1.5708, 0)        # joint_gripper
]

scaled_joint_angles = []
for i, angle in enumerate(joint_angles):
    lower_limit, upper_limit = joint_limits[i]
    scaled_angle = max(lower_limit, min(upper_limit, angle))
    scaled_joint_angles.append(scaled_angle)
print("Joint Angles in Robot Coordinate System:\n", scaled_joint_angles)

for i in range(len(scaled_joint_angles)):
    p.resetJointState(robot_id, i, scaled_joint_angles[i])

Joint Angles in Robot Coordinate System:
 (0.41986692368744477, 0.4662828109100441, 0.1739684877707861, 0.3208293913288225, 0.04130570759719363, 0.0)
Joint Angles in Robot Coordinate System:
 [0.41986692368744477, 0.4662828109100441, 0.1739684877707861, 0.3208293913288225, 0.04130570759719363, 0]


In [4]:
# move to grasp pose

joint_angles_degrees = np.degrees(joint_angles)
print("Joint Angles After Degree:\n", joint_angles_degrees)
joint_angles_real= 90 - joint_angles_degrees
joint_angles_real[0] -= 90
print("Joint Angles After Trans:\n", joint_angles_real)
for _ in range(record_time_s * fps):
    start_time = time.perf_counter()
    follower_pos = robot.follower_arms["main"].read("Present_Position")  
    robot.follower_arms["main"].write("Goal_Position", joint_angles_real)

    dt_s = time.perf_counter() - start_time
    busy_wait(1 / fps - dt_s)


Joint Angles After Degree:
 [24.05660268 26.71603712  9.96766012 18.38217007  2.36664272  0.        ]
Joint Angles After Trans:
 [-24.05660268  63.28396288  80.03233988  71.61782993  87.63335728
  90.        ]


In [5]:
# grasp and lift up
robot.follower_arms["main"].write("Goal_Position", 0, "gripper")


In [6]:
lift_position_robot = target_position_robot
lift_position_robot[2] += 0.1
joint_angles = p.calculateInverseKinematics(robot_id, 5, lift_position_robot)
joint_angles_real = sim2real(joint_angles)
joint_angles_real[-1] = 0
robot.follower_arms["main"].write("Goal_Position", joint_angles_real)

In [7]:
robot.disconnect()